<a href="https://colab.research.google.com/github/hilaltekgz/NLP/blob/main/muse_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
!pip install tensorflow-text

     |████████████████████████████████| 4.9 MB 9.5 MB/s 


In [ ]:
#!pip install sentence_transformers
#!pip install datasets
!pip install scipy

In [ ]:
"""
This example loads the pre-trained SentenceTransformer model 'nli-distilroberta-base-v2' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.

Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
import scipy
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv


from datasets import load_dataset
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import nltk
import os

from scipy.spatial import distance
from sklearn.model_selection import train_test_split

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

#Check if dataset exsist. If not, download and extract  it
#sts_dataset_path = 'datasets/stsbenchmark.tsv.gz'

DATA_IN_PATH = '/content/'

TRAIN_STS_DF = "/content/stsb_tr.tsv"


data  = pd.read_csv(TRAIN_STS_DF, header=0, delimiter = '\t', quoting = 3, keep_default_na=False)

sts_dataset  , dev_df = train_test_split(data, test_size=0.33 )
sts_test  , sts_dev = train_test_split(dev_df, test_size=0.33 )

sts_dev = sts_dev[[isinstance(s, str) for s in sts_dev['sentence2']]]
# Read the dataset
#model_name = 'dbmdz/bert-base-turkish-cased'
#model_name = 'bert-base-multilingual-cased'
#model_name = 'sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens'
#model_name = 'sentence-transformers/LaBSE'
model_name = 'universal-sentence-encoder-multilingual'
train_batch_size = 16
num_epochs = 4
model_save_path = 'output/training_stsbenchmark_continue_traininglabse-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


# model = SentenceTransformer('sentence-transformers/LaBSE')
# Load a pre-trained sentence transformer model
# model = SentenceTransformer(model_name)
#model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")
  
# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark train dataset")





sts_data = sts_test

def run_sts_benchmark(batch):
  sts_encode1 = tf.nn.l2_normalize(model(tf.constant(batch['sentence1'].tolist())), axis=1)
  sts_encode2 = tf.nn.l2_normalize(model(tf.constant(batch['sentence2'].tolist())), axis=1)
  cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
  clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
  scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
  """Returns the similarity scores"""
  return scores

dev_scores = sts_data['score'].tolist()
scores = []
for batch in np.array_split(sts_data, 10):
  scores.extend(run_sts_benchmark(batch))

pearson_correlation = scipy.stats.pearsonr(scores, dev_scores)
print('Pearson correlation coefficient = {0}\np-value = {1}'.format(
    pearson_correlation[0], pearson_correlation[1]))
spearman_correlation  = scipy.stats.spearmanr(scores, dev_scores)
print('Spearmanr correlation coefficient = {0}\np-value = {1}'.format(
    spearman_correlation[0], spearman_correlation[1]))

INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3'.
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3, Total size: 334.32MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3'.


Pearson correlation coefficient = 0.7603074150367316
p-value = 0.0
Spearmanr correlation coefficient = 0.746038909961859
p-value = 0.0
